In [ ]:
!pip install -qU diffusers transformers accelerate

# DiffEdit

Traditional image editing requires providing a mask of the area to be edited.

**DiffEdit** automatically generates the mask based on a text query, making it easier overall to create a mask without image editing software.

The DiffEdit algorithm works in three steps:
1. the diffusion model denoises an image conditioned on some query text and reference text which produces different noise estimates for different areas of the image; the difference is used to infer a mask to identify which area of the image needs to be changed to match the query text
2. the input image is encoded into latent space with DDIM
3. the latents are decoded with the diffusion model conditioned on the text query, using the mask as a guide such that pixels outside the mask remain the same as in the input image

The `StableDiffusionDiffEditPipeline` requires an image mask and a set of partially inverted latents.

The image mask is generated from the `generate_mask()`, and includes two parameters, `source_prompt` and `target_prompt`. These parameters determine what to edit in the image.

In [ ]:
source_prompt = 'a bowl of fruits'
target_prompt = 'a bowl of pears'

The partially inverted latents are generated from the `invert()` function, and it is generally a good idea to include a `prompt` or caption describing the image to help guide the inverse latent sampling process. The caption can be our `source_prompt`.

In [ ]:
from diffusers import DDIMScheduler, DDIMInverseScheduler, StableDiffusionDiffEditPipeline
import torch

pipeline = StableDiffusionDiffEditPipeline.from_pretrained(
    'stabilityai/stable-diffusion-2-1',
    torch_dtype=torch.float16,
    use_safetensors=True,
)
pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)
pipeline.inverse_scheduler = DDIMInverseScheduler.from_config(pipeline.scheduler.config)
pipeline.enable_model_cpu_offload()
pipeline.enable_vae_slicing()

In [ ]:
from diffusers.utils import load_image, make_image_grid

img_url = "https://github.com/Xiang-cd/DiffEdit-stable-diffusion/raw/main/assets/origin.png"
raw_image = load_image(img_url).resize((768, 768))
raw_image

Use the `generate_mask` function to generate the image mask.

In [ ]:
from PIL import Image

source_prompt = 'a bowl of fruits'
target_prompt = 'a basket of pears'

mask_image = pipeline.generate_mask(
    image=raw_image,
    source_prompt=source_prompt,
    target_prompt=target_prompt
)

mask_image = Image.fromarray(
    (mask_image.squeeze()*255).astype('unit8'),
    'L'
).resize((768, 768))

Next, create the inverted latents and pass it to a caption describing the image:

In [ ]:
inv_latents = pipeline.invert(
    prompt=source_prompt,
    image=raw_image,
).latents

Finally, pass the image mask and inverted latents to the pipeline. The `target_prompt` becomes the `prompt` now, and the `source_prompt` is used as the `negative_prompt`:

In [ ]:
output_image = pipeline(
    prompt=target_prompt,
    negative_prompt=source_prompt,
    mask_image=mask_image,
    image_latents=inv_latents
).images[0]
make_image_grid([raw_image, mask_image, output_image], rows=1, cols=3)

## Generate source and target embeddings

The source and target embeddings can be automatically generated with the `Flan-T5` model.

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration
import torch

tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-large')
model = T5ForConditionalGeneration.from_pretrained(
    'google/flan-t5-large',
    torch_dtype=torch.float16,
    device_map='auto'
)

In [ ]:
source_concept = 'bowl'
target_concept = 'basket'

source_text = f"Provide a caption for images containing a {source_concept}"
target_text = f"Provide a caption for images containing a {target_concept}"
print(f"Source text: {source_text}")
print(f"Target text: {target_text}")

In [ ]:
@torch.no_grad()
def generate_prompts(input_prompt):
    input_ids = tokenizer(
        input_prompt,
        return_tensors='pt'
    ).input_ids.to('cuda')

    outputs = model.generate(
        input_ids,
        temperature=0.8,
        num_return_sequences=16,
        do_sample=True,
        max_new_tokens=128,
        top_k=10
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

source_prompts = generate_prompts(source_text)
target_prompts = generate_prompts(target_text)
print(f"Source prompts: {source_prompts}")
print(f"Target prompts: {target_prompts}")

In [ ]:
from diffusers import StableDiffusionDiffEditPipeline
import torch

pipeline = StableDiffusionDiffEditPipeline.from_pretrained(
    'stabilityai/stable-diffusion-2-1',
    torch_dtype=torch.float16,
    use_safetensors=True,
)
pipeline.enable_model_cpu_offload()
pipeline.enable_vae_slicing()

In [ ]:
@torch.no_grad()
def embed_prompts(sentences, tokenizer, text_encoder, device='cuda'):
    embeddings = []
    for sent in sentences:
        text_inputs = tokenizer(
            sent,
            padding='max_length',
            max_length=tokenizer.model_max_length,
            truncation=True,
            return_tensors='pt'
        )
        text_input_ids = text_inputs.input_ids
        prompt_embed = text_encoder(
            text_input_ids.to(device),
            attention_mask=None
        )[0]
        embeddings.append(prompt_embed)

    return torch.concatenate(embeddings, dim=0).mean(dim=0).unsqueeze(0)


source_embeds = embed_prompts(source_prompts, tokenizer, pipeline.text_encoder)
target_embeds = embed_prompts(target_prompts, tokenizer, pipeline.text_encoder)

In [ ]:
from diffusers import DDIMScheduler, DDIMInverseScheduler
from diffusers.utils import load_image, make_image_grid
from PIL import Image

pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)
pipeline.inverse_scheduler = DDIMInverseScheduler.from_config(pipeline.scheduler.config)

In [ ]:
img_url = "https://github.com/Xiang-cd/DiffEdit-stable-diffusion/raw/main/assets/origin.png"
raw_image = load_image(img_url).resize((768, 768))

mask_image = pipeline.generate_mask(
    image=raw_image,
    source_prompt=source_embeds,
    target_prompt=target_embeds,
)

inv_latents = pipeline.invert(
    prompt_embeds=source_embeds,
    image=raw_image,
).latents

output_image = pipeline(
    mask_image=mask_image,
    image_latents=inv_latents,
    prompt_embeds=target_embeds,
    negative_prompt_embeds=source_embeds,
).images[0]

make_image_grid([raw_image, mask_image, output_image], rows=1, cols=3)

## Generate a caption for inversion

While we can use the `source_prompt` as a caption to help generate the partially inverted latents, we can also use the `BLIP` model to automatically generate a caption.

In [ ]:
from transformers import BlipForConditionalGeneration, BlipProcessor
import torch

processor = BlipProcessor.from_pretrained('Salesforce/blip-image-captioning-base')
model = BlipForConditionalGeneration.from_pretrained(
    'Salesforce/blip-image-captioning-base',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

In [ ]:
@torch.no_grad()
def generate_caption(images, caption_generator, caption_processor):
    text = 'a photograph of'

    inputs = caption_processor(
        images,
        text,
        return_tensors='pt'
    ).to(device='cuda', dtype=caption_generator.dtype)

    caption_generator.to('cuda')

    outputs = caption_generator.generate(
        **inputs,
        max_new_tokens=128
    )

    # offload caption generator
    caption_generator.to('cpu')

    caption = caption_processor.batch_decode(
        outputs,
        skip_special_tokens=True
    )[0]
    return caption

In [ ]:
from diffusers.utils import load_image

img_url = "https://github.com/Xiang-cd/DiffEdit-stable-diffusion/raw/main/assets/origin.png"
raw_image = load_image(img_url).resize((768, 768))

caption = generate_caption(raw_image, model, processor)
caption